In [1]:
cd ../

c:\Users\eduar\Documents\usp\AMST\ml_timeseries_usp


In [2]:
from src.data.make_dataset import DatasetCreator
from src.data.make_hierarchical_dataset import DatasetHierarchicalAggregator
from src.data.refine_dataset import HierarchicalTimeSeriesOutlierRemover
from src.data.split import hierarchical_train_test_split
from src.features.build_features import FeaturesBuilder
from src.config import load_config
from src.models.tuning import ModelTuning

config = load_config()

df = DatasetCreator(config).run() 

if config['exogen_features'] == 'true':
    df = FeaturesBuilder(df,config).run()
else:
    pass

Y_df,S_df,tags = DatasetHierarchicalAggregator(config,df).run()

train,test = hierarchical_train_test_split(Y_df)

2025-11-28 17:33:44,082 - INFO - Config carregado de c:\Users\eduar\Documents\usp\AMST\ml_timeseries_usp\config.yaml
2025-11-28 17:33:44,697 - INFO - Config carregado de c:\Users\eduar\Documents\usp\AMST\ml_timeseries_usp\config.yaml


Filtrando nível: ['total', 'centro_distribuicao'] (>= 42 obs)
   → 3 combinações → 3 mantidas (100.0%)
Filtrando nível: ['total'] (>= 42 obs)
   → 1 combinações → 1 mantidas (100.0%)


2025-11-28 17:34:51,271 - INFO - Agregação hierárquica concluída.


tags salvo com joblib → 100% fiel ao original
Tamanho: 0.4 KB
Train: 2022-01-01 → 2025-01-01 (37 meses)
Test (backtest): 2025-02-01 → 2025-07-01 (6 meses)


In [3]:
models = config['models']

for model in models:
    is_enable = models[model]['enabled']
    if is_enable:
        if models[model]['type'] == 'mlforecast':
            model_name = models[model]['regressor']
            fixed_params = models[model]['fixed_params']
            param_space = config['parameter_space'][model]
            mlforecast_params = config['modeling']['mlforecast']

            cv = ModelTuning(df=train,config=config,model_name=model_name,fixed_params=fixed_params,param_space=param_space,mlforecast_params=mlforecast_params)
            study = cv.run()
 #      else:
 #          model = train_model(params)
 #      save_model(model)
 #      save_metrics(metrics)
 #  else:
 #      pass

[I 2025-11-28 17:34:51,396] A new study created in memory with name: no-name-d72ab085-a340-44fe-aff7-8d133f931785
[I 2025-11-28 17:34:52,001] Trial 0 finished with value: 0.3844696633807856 and parameters: {'learning_rate': 0.03574712922600244, 'feature_fraction': 0.9852142919229748, 'bagging_fraction': 0.9195981825434215, 'lambda_l1': 0.0006155564318973012, 'lambda_l2': 1.77071686435378e-07, 'num_leaves': 16, 'max_depth': 3, 'min_data_in_leaf': 18, 'n_estimators': 1923}. Best is trial 0 with value: 0.3844696633807856.
[I 2025-11-28 17:34:52,665] Trial 1 finished with value: 0.2584687839615718 and parameters: {'learning_rate': 0.11114989443094977, 'feature_fraction': 0.7061753482887407, 'bagging_fraction': 0.9909729556485982, 'lambda_l1': 0.04566054873446119, 'lambda_l2': 4.997040685255803e-07, 'num_leaves': 18, 'max_depth': 4, 'min_data_in_leaf': 9, 'n_estimators': 1717}. Best is trial 1 with value: 0.2584687839615718.
[I 2025-11-28 17:34:53,160] Trial 2 finished with value: 0.3956154

In [4]:
study.best_params

{'learning_rate': 0.24870647064013596,
 'feature_fraction': 0.84850097717473,
 'bagging_fraction': 0.9806304543244749,
 'lambda_l1': 1.901941965182528e-06,
 'lambda_l2': 2.1404240108978166e-07,
 'num_leaves': 39,
 'max_depth': 9,
 'min_data_in_leaf': 9,
 'n_estimators': 2849}

In [6]:
study.best_value

0.24036374675934324

In [5]:
study.best_trial

FrozenTrial(number=52, state=<TrialState.COMPLETE: 1>, values=[0.24036374675934324], datetime_start=datetime.datetime(2025, 11, 28, 17, 35, 36, 480997), datetime_complete=datetime.datetime(2025, 11, 28, 17, 35, 37, 268350), params={'learning_rate': 0.24870647064013596, 'feature_fraction': 0.84850097717473, 'bagging_fraction': 0.9806304543244749, 'lambda_l1': 1.901941965182528e-06, 'lambda_l2': 2.1404240108978166e-07, 'num_leaves': 39, 'max_depth': 9, 'min_data_in_leaf': 9, 'n_estimators': 2849}, user_attrs={'config': {'model_params': {'objective': 'poisson', 'verbosity': -1, 'n_jobs': -1, 'learning_rate': 0.24870647064013596, 'feature_fraction': 0.84850097717473, 'bagging_fraction': 0.9806304543244749, 'lambda_l1': 1.901941965182528e-06, 'lambda_l2': 2.1404240108978166e-07, 'num_leaves': 39, 'max_depth': 9, 'min_data_in_leaf': 9, 'n_estimators': 2849}, 'mlf_init_params': {'lags': [1, 2, 3, 4, 5, 6, 12], 'lag_transforms': {1: [ExpandingMean()], 6: [RollingMean(window_size=3), RollingMea

In [37]:
study.

[<StudyDirection.MINIMIZE: 1>]

In [6]:
cv = HierarchicalTuning(config,train)

models_cfg = config['models']

for model in models_cfg:
    if models_cfg[model]['type'] == 'mlforecast':
        best_param =

{'enabled': True, 'type': 'mlforecast', 'regressor': 'LGBMRegressor', 'fixed_params': {'objective': 'poisson', 'boosting_type': 'gbdt', 'metric': 'mape', 'bagging_freq': 1, 'force_col_wise': True, 'verbose': -1, 'n_jobs': -1, 'seed': 42, 'deterministic': True}}
{'enabled': True, 'type': 'mlforecast', 'regressor': 'CatBoostRegressor', 'fixed_params': {'loss_function': 'Poisson', 'eval_metric': 'MAPE', 'thread_count': -1, 'verbose': False, 'random_seed': 42, 'cat_features': 'auto', 'allow_writing_files': False}}
{'enabled': True, 'type': 'mlforecast', 'regressor': 'XGBRegressor', 'fixed_params': {'objective': 'reg:squarederror', 'eval_metric': 'mape', 'n_jobs': -1, 'seed': 42, 'tree_method': 'hist', 'enable_categorical': True}}
{'enabled': True, 'type': 'statsforecast', 'fixed_params': {'season_length': 12, 'approximation': True}}
{'enabled': True, 'type': 'statsforecast', 'fixed_params': {'season_length': 12, 'model': 'ZZZ'}}
{'enabled': True, 'type': 'statsforecast', 'fixed_params': {'

In [ ]:
HierarchicalMLTunning

In [18]:
# src/models/tuning.py — VERSÃO FINAL 2025 — TUDO FUNCIONA
import optuna
import pandas as pd
from mlforecast import MLForecast
from statsforecast import StatsForecast

# Mapeamento string → objeto real (o coração do sistema)
from src.utils.presets import build_lag_transforms, build_target_transforms
from src.utils.model_registry import get_model_class

class HierarchicalTuning:
    def __init__(self, config: dict, df: pd.DataFrame):
        self.config = config
        self.df = df.copy()
        self.models_cfg = config["models"]
        self.ml_cfg = config["modeling"]["mlforecast"]
        self.presets = self.ml_cfg["presets"]
        self.tuning_cfg = self.ml_cfg["tuning"]

    def objective(self, trial):
        # === 1. ESCOLHE O MODELO ===
        model_name = trial.suggest_categorical(
            "model",
            [name for name, cfg in self.models_cfg.items() if cfg.get("enabled", False)]
        )

        # === 2. RODA O MODELO ESCOLHIDO ===
        if self.models_cfg[model_name]["type"] == "mlforecast":
            return self._tune_mlforecast_model(trial, model_name)
        else:  # StatsForecast
            return self._evaluate_statsforecast()

    def _tune_mlforecast_model(self, trial, model_name: str):
        cfg = self.models_cfg[model_name]
        Regressor = get_model_class(cfg["regressor"])

        # === Hiperparâmetros fixos + tuning específico do modelo ===
        params = cfg["fixed_params"].copy()
        search_space = self.config["parameter_space"].get(model_name, {})

        for param, space in search_space.items():
            if "log" in space and space["log"]:
                if space.get("type") == "int" or "int" in param:
                    params[param] = trial.suggest_int(param, space["low"], space["high"], log=True)
                else:
                    params[param] = trial.suggest_float(param, space["low"], space["high"], log=True)
            else:
                if space.get("type") == "int" or "int" in param:
                    params[param] = trial.suggest_int(param, space["low"], space["high"])
                else:
                    params[param] = trial.suggest_float(param, space["low"], space["high"])

        # === PRESETS DE LAG E TARGET ===
        if self.tuning_cfg["search_lag_preset"]:
            lag_preset = trial.suggest_categorical("lag_preset", list(self.presets["lag_strategies"].keys()))
        else:
            lag_preset = self.tuning_cfg["active_lag_preset"]

        if self.tuning_cfg["search_target_preset"]:
            target_preset = trial.suggest_categorical("target_preset", list(self.presets["target_strategies"].keys()))
        else:
            target_preset = self.tuning_cfg["active_target_preset"]

        lag_cfg = self.presets["lag_strategies"][lag_preset]
        target_names = self.presets["target_strategies"][target_preset]

        # === MONTA O MLFORECAST ===
        fcst = MLForecast(
            models={model_name: Regressor(**params)},
            freq=self.ml_cfg["freq"],
            lags=lag_cfg["lags"],
            lag_transforms=build_lag_transforms(lag_cfg.get("transforms", {})),
            target_transforms=build_target_transforms(target_names),
            date_features=self.presets["date_features"]["basic"],  # ou tunar
        )

        # === CROSS VALIDATION ===
        cv_results = fcst.cross_validation(
            df=self.df,
            n_windows=self.ml_cfg["n_windows"],
            h=self.ml_cfg["h"],
            step_size=self.ml_cfg["step_size"],
            id_col=self.ml_cfg["id_col"],
            time_col=self.ml_cfg["time_col"],
            target_col=self.ml_cfg["target_col"],
        )

        score = cv_results[model_name].mean()
        trial.set_user_attr("model", model_name)
        trial.set_user_attr("lag_preset", lag_preset)
        trial.set_user_attr("target_preset", target_preset)

        return score

    def _evaluate_statsforecast(self):
        from statsforecast.models import AutoARIMA, AutoETS, AutoTheta, SeasonalNaive

        models = []
        for name, params in self.models_cfg["StatsForecast"]["models"].items():
            model_class = globals()[name]
            models.append(model_class(**params))

        sf = StatsForecast(
            df=self.df,
            models=models,
            freq="MS",
            n_jobs=-1,
        )

        cv = sf.cross_validation(
            df=self.df,
            n_windows=self.ml_cfg["n_windows"],
            h=self.ml_cfg["h"],
            step_size=self.ml_cfg["step_size"],
        )
        return cv.mean().mean()

    def run(self):
        study = optuna.create_study(
            direction="minimize",
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=20)
        )

        study.optimize(self.objective, n_trials=self.ml_cfg["validation"]["n_trials"])

        best = study.best_trial
        print("\n" + "="*60)
        print("MELHOR CONFIGURAÇÃO ENCONTRADA")
        print("="*60)
        print(f"Modelo: {best.user_attrs['model']}")
        print(f"lag_preset: {best.user_attrs.get('lag_preset', 'N/A')}")
        print(f"target_preset: {best.user_attrs.get('target_preset', 'N/A')}")
        print(f"sMAPE CV: {best.value:.4f}%")
        print(f"Trial: {best.number}")
        print("="*60)

        return study

In [22]:
fcst = MLForecast(
    models = cv.LGBMRegressor(objective='poisson',
            learning_rate=0.03,
            num_leaves=16,
            max_depth=5,
            min_data_in_leaf=6,
            feature_fraction=0.8,
            bagging_fraction=0.8,
            lambda_l1=0.5,
            lambda_l2=1.0,
            verbosity=-1,
            n_jobs=4),
    freq='MS',
    lags=range(1,13,1),
    lag_transforms={
        1:[ExpandingMean()],
        2:[RollingMean(window_size=3)],
    },
    
    date_features = ['month','year'],
    num_threads = 1
)

NameError: name 'lgbm' is not defined

In [21]:
MLForecast?

Init signature:
MLForecast(
    models: Union[sklearn.base.BaseEstimator, List[sklearn.base.BaseEstimator], Dict[str, sklearn.base.BaseEstimator]],
    freq: Union[int, str],
    lags: Optional[Iterable[int]] = None,
    lag_transforms: Optional[Dict[int, List[Union[Callable, Tuple[Callable, Any]]]]] = None,
    date_features: Optional[Iterable[Union[str, Callable]]] = None,
    num_threads: int = 1,
    target_transforms: Optional[List[Union[mlforecast.target_transforms.BaseTargetTransform, mlforecast.target_transforms._BaseGroupedArrayTargetTransform]]] = None,
    lag_transforms_namer: Optional[Callable] = None,
)
Docstring:      <no docstring>
Init docstring:
Forecasting pipeline

Parameters
----------
models : regressor or list of regressors
    Models that will be trained and used to compute the forecasts.
freq : str or int or pd.offsets.BaseOffset
    Pandas offset, pandas offset alias, e.g. 'D', 'W-THU' or integer denoting the frequency of the series.
lags : list of int, option

In [19]:
cv = HierarchicalTuning(config,df)
cv.run()

[I 2025-11-27 17:01:44,585] A new study created in memory with name: no-name-89c683e6-04ee-43d4-a912-68a9bf7bc5c2
[W 2025-11-27 17:01:44,591] Trial 0 failed with parameters: {'model': 'CatBoost', 'learning_rate': 0.01700037298921102, 'iterations': 716.0849144555755, 'depth': 4.464668897345596, 'l2_leaf_reg': 7.348118405270449, 'bagging_temperature': 0.6011150117432088, 'lag_preset': 'short_seasonal', 'target_preset': 'standardize'} because of the following error: KeyError('date_features').
Traceback (most recent call last):
  File "c:\Users\eduar\AppData\Local\anaconda3\envs\or-env\lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\eduar\AppData\Local\Temp\ipykernel_31972\1910238161.py", line 29, in objective
    return self._tune_mlforecast_model(trial, model_name)
  File "C:\Users\eduar\AppData\Local\Temp\ipykernel_31972\1910238161.py", line 74, in _tune_mlforecast_model
    date_features=self.presets["date_feature

KeyError: 'date_features'

In [6]:
def objective(trial):
        # === 1. ESCOLHE O MODELO ===
        model_name = trial.suggest_categorical(
            "model",
            [name for name, cfg in cv.models_cfg.items() if cfg.get("enabled", False)]
        )

        # === 2. RODA O MODELO ESCOLHIDO ===
        if cv.models_cfg[model_name]["type"] == "mlforecast":
            return cv._tune_mlforecast_model(trial, model_name)
        else:  # StatsForecast
            return cv._evaluate_statsforecast()

In [15]:
cfg = cv.models_cfg['LightGBM']
Regressor = get_model_class(cfg["regressor"])
params = cfg["fixed_params"].copy()
search_space = cv.config["parameter_space"].get('LightGBM', {})
cv.tuning_cfg["search_lag_preset"]


True

In [ ]:
def _tune_mlforecast_model(trial, model_name: str):
        cfg = cv.models_cfg[model_name]
        Regressor = get_model_class(cfg["regressor"])

        # === Hiperparâmetros fixos + tuning específico do modelo ===
        params = cfg["fixed_params"].copy()
        search_space = cv.config["parameter_space"].get(model_name, {})

        for param, space in search_space.items():
            if "log" in space and space["log"]:
                if space.get("type") == "int" or "int" in param:
                    params[param] = trial.suggest_int(param, space["low"], space["high"], log=True)
                else:
                    params[param] = trial.suggest_float(param, space["low"], space["high"], log=True)
            else:
                if space.get("type") == "int" or "int" in param:
                    params[param] = trial.suggest_int(param, space["low"], space["high"])
                else:
                    params[param] = trial.suggest_float(param, space["low"], space["high"])

        # === PRESETS DE LAG E TARGET ===
        if cv.tuning_cfg["search_lag_preset"]:
            lag_preset = trial.suggest_categorical("lag_preset", list(cv.presets["lag_strategies"].keys()))
        else:
            lag_preset = cv.tuning_cfg["active_lag_preset"]

        if cv.tuning_cfg["search_target_preset"]:
            target_preset = trial.suggest_categorical("target_preset", list(cv.presets["target_strategies"].keys()))
        else:
            target_preset = cv.tuning_cfg["active_target_preset"]

        lag_cfg = cv.presets["lag_strategies"][lag_preset]
        target_names = cv.presets["target_strategies"][target_preset]

        # === MONTA O MLFORECAST ===
        fcst = MLForecast(
            models={model_name: Regressor(**params)},
            freq=cv.ml_cfg["freq"],
            lags=lag_cfg["lags"],
            lag_transforms=build_lag_transforms(lag_cfg.get("transforms", {})),
            target_transforms=build_target_transforms(target_names),
            date_features=cv.presets["date_features"]["basic"],  # ou tunar
        )

        # === CROSS VALIDATION ===
        cv_results = fcst.cross_validation(
            df=cv.df,
            n_windows=cv.ml_cfg["n_windows"],
            h=cv.ml_cfg["h"],
            step_size=cv.ml_cfg["step_size"],
            id_col=cv.ml_cfg["id_col"],
            time_col=cv.ml_cfg["time_col"],
            target_col=cv.ml_cfg["target_col"],
        )

        score = cv_results[model_name].mean()
        trial.set_user_attr("model", model_name)
        trial.set_user_attr("lag_preset", lag_preset)
        trial.set_user_attr("target_preset", target_preset)

        return score

In [16]:
MLForecast?

Init signature:
MLForecast(
    models: Union[sklearn.base.BaseEstimator, List[sklearn.base.BaseEstimator], Dict[str, sklearn.base.BaseEstimator]],
    freq: Union[int, str],
    lags: Optional[Iterable[int]] = None,
    lag_transforms: Optional[Dict[int, List[Union[Callable, Tuple[Callable, Any]]]]] = None,
    date_features: Optional[Iterable[Union[str, Callable]]] = None,
    num_threads: int = 1,
    target_transforms: Optional[List[Union[mlforecast.target_transforms.BaseTargetTransform, mlforecast.target_transforms._BaseGroupedArrayTargetTransform]]] = None,
    lag_transforms_namer: Optional[Callable] = None,
)
Docstring:      <no docstring>
Init docstring:
Forecasting pipeline

Parameters
----------
models : regressor or list of regressors
    Models that will be trained and used to compute the forecasts.
freq : str or int or pd.offsets.BaseOffset
    Pandas offset, pandas offset alias, e.g. 'D', 'W-THU' or integer denoting the frequency of the series.
lags : list of int, option

In [5]:
study = cv.run()

[I 2025-11-27 16:43:40,506] A new study created in memory with name: no-name-1dda418b-3a7c-4067-a67b-73749e523f96
c:\Users\eduar\AppData\Local\anaconda3\envs\or-env\lib\site-packages\utilsforecast\processing.py:757: UserWarning: The following series are too short for the window and will be dropped: ['060d1444b749...fed65fd19b844', 'd91fc0ba7e55...1ff568dc80696', '1a15f0ded931...428e3192301c2', '24554ce00cef...2e394d886fa37', '28f6a1709cd5...5397ee7585bf4', '2e0d77ccf256...99ce24db38294', ...]
  warnings.warn(
[W 2025-11-27 16:43:42,647] Trial 0 failed with parameters: {'model': 'CatBoost', 'learning_rate': 0.01700037298921102, 'iterations': 716.0849144555755, 'depth': 4.464668897345596, 'l2_leaf_reg': 7.348118405270449, 'bagging_temperature': 0.6011150117432088, 'lag_preset': 'short_seasonal', 'target_preset': 'standardize'} because of the following error: ValueError('centro_distribuicao is declared as a static feature but its values change over time. Please set the `static_features` a

ValueError: centro_distribuicao is declared as a static feature but its values change over time. Please set the `static_features` argument to indicate which features are static.
If all of your features are dynamic please set `static_features=[]`.

In [3]:
from mlforecast.target_transforms import Differences, LocalBoxCox, LocalStandardScaler
from mlforecast.lag_transforms import ExpandingMean,RollingMean
from mlforecast import MLForecast
from mlforecast.utils import PredictionIntervals
from mlforecast.optimization import mlforecast_objective
import optuna
import lightgbm as lgbm

In [5]:
# Mapeamento: string do YAML → objeto real
TRANSFORM_MAP = {
    "expanding_mean": ExpandingMean(),
    "rolling_mean_3": RollingMean(window_size=3),
    "rolling_mean_6": RollingMean(window_size=6),
    "rolling_mean_12": RollingMean(window_size=12),
    "differences_1": Differences([1]),
    "boxcox": LocalBoxCox(),
    "standard_scaler": LocalStandardScaler(),
}

def build_lag_transforms(transform_cfg: dict) -> dict:
    """Converte config do YAML em dict de objetos reais"""
    if not transform_cfg:
        return {}
    
    result = {}
    for lag_str, transform_names in transform_cfg.items():
        lag = int(lag_str)
        result[lag] = [TRANSFORM_MAP[name] for name in transform_names]
    return result

def build_target_transforms(transform_names: list) -> list:
    """Converte lista de strings em lista de objetos"""
    if not transform_names:
        return []
    return [TRANSFORM_MAP[name] for name in transform_names]

In [15]:
transforms = config['modeling']['mlforecast']['lag_presets']['only_seasonal']['transforms']

In [16]:
build_lag_transforms(transforms)

{}

In [ ]:
build_lag_transforms

In [ ]:
build_lag_transforms(lag_cfg["transforms"])

In [16]:
# Mapeamento: string do YAML → objeto real
TRANSFORM_MAP = {
    "expanding_mean": ExpandingMean(),
    "rolling_mean_3": RollingMean(window_size=3),
    "rolling_mean_6": RollingMean(window_size=6),
    "rolling_mean_12": RollingMean(window_size=12),
    "differences_1": Differences([1]),
    "boxcox": LocalBoxCox(),
    "standard_scaler": LocalStandardScaler(),
}

In [13]:
# 2. Treine normalmente
fcst = MLForecast(
    models=lgbm.LGBMRegressor(objective='poisson',
            learning_rate=0.03,
            num_leaves=16,
            max_depth=5,
            min_data_in_leaf=6,
            feature_fraction=0.8,
            bagging_fraction=0.8,
            lambda_l1=0.5,
            lambda_l2=1.0,
            verbosity=-1,
            n_jobs=4),
    freq='MS',
    lags=lags,
    lag_transforms=transforms,
    
    #date_features = ['month','year'],
    num_threads = 1)

fcst.fit(train, static_features=[])   # tudo dinâmico

AssertionError: 

In [7]:
lags = config['modeling']['mlforecast']['init_params']['lag_strategies']['short_and_seasonal']['lags']

In [10]:
transforms = config['modeling']['mlforecast']['init_params']['lag_strategies']['short_and_seasonal']['transforms']

In [15]:
RollingMean(window_size=3)

RollingMean(window_size=3)

In [11]:
transforms

{6: [{'RollingMean': {'window_size': 3}}],
 12: [{'RollingMean': {'window_size': 3}}]}

In [3]:
Y_df

,unique_id,ds,y
0,total,2022-01-01,24816
1,total,2022-02-01,27690
2,total,2022-03-01,20979
3,total,2022-04-01,16943
4,total,2022-05-01,16950
...,...,...,...
167,total/CD3,2025-03-01,4250
168,total/CD3,2025-04-01,4207
169,total/CD3,2025-05-01,4737
170,total/CD3,2025-06-01,4476
